In [2]:
from ThesisFunctions import *

In [14]:
print('Number of datapoints vs Number of minimas')
noise_level = 10
num_datapoints_list = [3, 4, 5, 6, 7]

# Initialize a list to store the results.
results_list = []

for num_datapoints in num_datapoints_list:
    trials_minimas = []  # Store the number of minima found in each trial for current num_datapoints.
    
    for _ in range(20):  # Perform 10 trials to average out the effects of randomness.
        # Generate data with specified number of datapoints and noise level.
        data = data_gen(num_datapoints, noise_level, 1, 1)
        x_i = np.array(data['Data'].values)
        t_i = np.array(data['Time'].values)
        
        # Calculate the Groebner basis.
        B = groeb(x_i, t_i)
                            
        # Find positive real roots.
        positive_roots_b = roots_symbolic(B[1])
            
        # Determine (x0, alpha) pairs.
        params = find_x0_alpha_pairs(B, positive_roots_b)
    
        # Evaluate the Hessian to check for minima.
        maxima_checks = evaluate_hessian_at_extremas(params, x_i, t_i)
    
        # Count the number of minima.
        num_minimas = sum(1 for _, is_true in maxima_checks if is_true)
        trials_minimas.append(num_minimas)
    
    # Calculate the average number of minima for the current number of data points.
    avg_minimas = np.mean(trials_minimas)
    results_list.append({'Number_of_Datapoints': num_datapoints, 'Average_Number_of_Minimas': avg_minimas})

# Convert results to a DataFrame for better visualization.
results_df = pd.DataFrame(results_list)

# Print the DataFrame to see the results.
print(results_df)


Number of datapoints vs Number of minimas
   Number_of_Datapoints  Average_Number_of_Minimas
0                     3                       0.60
1                     4                       0.95
2                     5                       1.20
3                     6                       1.25
4                     7                       1.25
